<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_NY_mobility_poverty_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 1.92 ms (started: 2021-04-12 05:45:47 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 47.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=34f55b4ea1db53619e32fe6242edc72f3cbb8074919cf792f9cbe57a27f632eb
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 86.7 ms (started: 2021-04-12 05:47:06 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 245 ms (started: 2021-04-12 05:47:08 +00:00)


In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.64 s (started: 2021-04-12 05:47:18 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 20.1 s (started: 2021-04-12 05:47:27 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 1.9 ms (started: 2021-04-12 05:47:50 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [ ]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 789 µs (started: 2021-04-12 00:27:40 +00:00)


In [16]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|                              8|
|360850170103|2020-02-03T00:00:...|         250|                          68|                             41|                             26|
|360050213021|2020-02-03T00:00:...|          92|                          26|                             11|                              6|
|36005

In [17]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 3.07 ms (started: 2021-04-12 05:52:39 +00:00)


In [18]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|       

In [19]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 3.31 ms (started: 2021-04-12 05:52:53 +00:00)


In [22]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 22.6 ms (started: 2021-04-12 05:57:29 +00:00)




---


demographic data






---



In [21]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050175002|     1110| 55.22522522522523|14.774774774774773|
|360050141001|     1445| 36.74740484429066| 19.16955017301038|
|360050145001|     2003| 39.47667804323095|  8.53719420868697|
|360050075002|     2731|22.812156719150494|4.5404613694617355|
|360050418001|      936|18.859649122807017| 9.081196581196581|
|360050125001|     2935| 43.24693042291951|12.742759795570699|
|360050127012|     1061| 43.73232799245994|18.378887841658813|
|360050054003|     1031| 57.12900096993211| 28.12803103782735|
|360050027011|     1364|  65.9090909090909|14.589442815249267|
|360050458001|     1168| 37.15753424657534| 2.910958904109589|
|360050224011|     1291|19.907048799380327|27.420604182804027|
|360050389001|     1143| 53.94265232974911|30.708661417322837|
|360050071002|     1661|           37.4375|11.499096929

In [28]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|poverty_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125| 19.22705314009662|
|360810384001|2020-02-03T00:00:...|     

In [ ]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM demographic_mobility').show()

+----------+
|count(cbg)|
+----------+
|      6369|
+----------+

time: 37.6 s (started: 2021-04-12 00:31:13 +00:00)


In [29]:
#Sort by poverty
df_mob_demo = spark.sql('SELECT * FROM demographic_mobility ORDER BY CAST(poverty_percentage AS float) DESC')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()

KeyboardInterrupt: ignored

time: 5min 17s (started: 2021-04-12 06:16:20 +00:00)


In [ ]:
# group by poverty range percentage



time: 4.39 ms (started: 2021-04-12 00:33:39 +00:00)


In [31]:
df_mob_demo_pandas = df_mob_demo.toPandas()
df_mob_demo_pandas

KeyboardInterrupt: ignored

time: 28.7 s (started: 2021-04-12 06:22:04 +00:00)


In [ ]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 343 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
result_mean_df = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,110, 10))).mean().reset_index()

time: 21.7 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
result_mean_df['poverty_percentage'] = result_mean_df['poverty_percentage'].astype(str)

time: 3.69 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
result_mean_df['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in result_mean_df['mean_completely_home_percentage']]
result_mean_df

,poverty_percentage,mean_completely_home_percentage,mean_full_time_work_percentage,mean_part_time_work_percentage,color
0,"(0, 10]",36.293477,5.604502,6.687025,#45a0d1
1,"(10, 20]",38.615480,5.112938,6.140845,#45a0d1
2,"(20, 30]",39.525652,4.905658,5.931445,#45a0d1
3,"(30, 40]",40.106181,4.759459,5.882565,#45a0d1
4,"(40, 50]",39.938050,4.762978,5.845302,#45a0d1
5,"(50, 60]",39.021866,4.812153,5.913782,#45a0d1
6,"(60, 70]",38.469605,5.566070,6.572136,#45a0d1
7,"(70, 80]",36.793549,6.209172,7.280926,#45a0d1
8,"(80, 90]",33.230229,5.034002,6.030464,#45a0d1
9,"(90, 100]",NaN,NaN,NaN,#ff4833


time: 28.4 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
count = result_mean_df.shape[0]
print("{}".format(count))

10
time: 2.7 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
viz_df = []
for col in ['mean_completely_home_percentage','mean_full_time_work_percentage','mean_part_time_work_percentage']:
        for c in range(0,count):
            viz_df.append({'poverty': c, 'column': col, 'value': result_mean_df[col][c]})
viz_df = alt.Data(values=viz_df)

time: 8.72 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
alt.Chart(result_mean_df).mark_bar(size=30).encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 31.7 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()



alt.Chart(...)

time: 30.6 ms (started: 2021-04-12 00:35:33 +00:00)


In [ ]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')

time: 7.6 ms (started: 2021-04-12 00:35:34 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_full_time_work_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_full_time_work_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 101 ms (started: 2021-04-12 00:35:34 +00:00)


In [ ]:
#
alt.Chart(result_mean_df).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_part_time_work_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_part_time_work_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

time: 53.4 ms (started: 2021-04-12 00:35:34 +00:00)
